In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from physnet import PhysNet, SelfAttention, PulseDataset

In [3]:
import torch
import torchvision.transforms as transforms

In [4]:
seq_len = 32 
model = PhysNet(seq_len)

In [5]:
model = torch.nn.DataParallel(model)

In [6]:
device = torch.device('cpu')
checkpoint = torch.load('pulsedetection.tar', map_location=device)

In [7]:
model.load_state_dict(checkpoint['state_dict'])
model.eval()

DataParallel(
  (module): PhysNet(
    (ConvBlock1): Sequential(
      (0): Conv3d(3, 16, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
      (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (ConvBlock2): Sequential(
      (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (ConvBlock3): Sequential(
      (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (ConvBlock4): Sequential(
      (0): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)


In [8]:
dummy_input = torch.cuda.FloatTensor(1, 3, 32, 120, 120)
torch.onnx.export(model.module, dummy_input, f='pulse.onnx', training=False, verbose=True, 
                  opset_version=9)

/home/d/miniconda3/envs/ONNX/lib/python3.8/site-packages/torch/onnx/utils.py:439: UserWarning: It is recommended that constant folding be turned off ('do_constant_folding=False') when exporting the model in training-amenable mode, i.e. with 'training=TrainingMode.TRAIN' or 'training=TrainingMode.PRESERVE' (when model is in training mode). Otherwise, some learnable model parameters may not translate correctly in the exported ONNX model because constant folding mutates model parameters. Please consider turning off constant folding or setting the training=TrainingMode.EVAL.
  warnings.warn(


Exported graph: graph(%input.1 : Float(1, 3, 32, 120, 120, strides=[1382400, 460800, 14400, 120, 1], requires_grad=0, device=cuda:0),
      %ConvBlock1.0.weight : Float(16, 3, 1, 5, 5, strides=[75, 25, 25, 5, 1], requires_grad=1, device=cuda:0),
      %ConvBlock1.0.bias : Float(16, strides=[1], requires_grad=1, device=cuda:0),
      %ConvBlock1.1.weight : Float(16, strides=[1], requires_grad=1, device=cuda:0),
      %ConvBlock1.1.bias : Float(16, strides=[1], requires_grad=1, device=cuda:0),
      %ConvBlock1.1.running_mean : Float(16, strides=[1], requires_grad=0, device=cuda:0),
      %ConvBlock1.1.running_var : Float(16, strides=[1], requires_grad=0, device=cuda:0),
      %ConvBlock2.0.weight : Float(32, 16, 3, 3, 3, strides=[432, 27, 9, 3, 1], requires_grad=1, device=cuda:0),
      %ConvBlock2.0.bias : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %ConvBlock2.1.weight : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %ConvBlock2.1.bias : Float(32, stri

/home/d/miniconda3/envs/ONNX/lib/python3.8/site-packages/torch/onnx/symbolic_helper.py:1135: UserWarning: ONNX export mode is set to inference mode, but operator dropout is set to training  mode. The operators will be exported in training , as specified by the functional operator.
  warnings.warn(
/home/d/miniconda3/envs/ONNX/lib/python3.8/site-packages/torch/onnx/symbolic_opset9.py:2384: UserWarning: Dropout is a training op and should not be exported in inference mode. For inference, make sure to call eval() on the model and to export it with param training=False.
  warnings.warn(


In [9]:
### DEBUG Uncaught 
# (in promise) TypeError: cannot resolve operator 'ConvTranspose' with opsets: ai.onnx v9

In [10]:
import onnx
model2 = onnx.load('pulse.onnx')

In [11]:
onnx.checker.check_model(model2)

In [12]:
onnx.save_model(model2, 'pulse2.onnx')

In [13]:
from torch.onnx import utils as onnx_utils

torch_script_graph, unconvertible_ops = onnx_utils.unconvertible_ops(
    model, dummy_input, opset_version=16)

print(set(unconvertible_ops))

{'aten::adaptive_avg_pool3d'}


/home/d/miniconda3/envs/ONNX/lib/python3.8/site-packages/torch/onnx/symbolic_opset12.py:57: UserWarning: Dropout is a training op and should not be exported in inference mode. For inference, make sure to call eval() on the model and to export it with param training=False.
  warnings.warn(
